In [1]:
# for Raster files 'tiff' manipulation
import rasterio

# libraries for array manipulation
import numpy as np
import pandas as pd

In [2]:
def readTiffBand(tiff_file, band_number):
    """
    Reading the specific band from the 'tiff' file.
    """
    with rasterio.open(tiff_file) as dataset:
        band = dataset.read(band_number)
    return band

In [3]:
def calculateNDVI(band_red, band_nir):
    """
    Calculating NDVI index given the red and near infrared band.
    """
    
    # In case you don't have cero values over the bands
    #ndvi = (band_nir - band_red) / (band_nir + band_red)
    
    # Cero values won't be valid
    element1 = (band_nir - band_red)
    element2 = (band_nir + band_red)
    
    ndvi = np.divide(element1, element2, out=np.zeros_like(element1), where=element2!=0)
    
    return ndvi

In [4]:
def calculateNDWI(band_green, band_nir):
    """
    Calculating NDWI index given the green and near infrared band.
    """
    
    # In case you don't have cero values over the bands
    #ndwi = (band_green - band_nir) / (band_green + band_nir)
    
    
    # Cero values won't be valid
    element1 = (band_green - band_nir)
    element2 = (band_green + band_nir)
    
    ndwi = np.divide(element1, element2, out=np.zeros_like(element1), where=element2!=0)
    
    return ndwi

In [5]:
def write_index_to_tiff(index_array, output_tiff_file, tiff_metadata):
    """
    Write the computed index array to the new 'tiff' file with the same metadata as the input 'tiff' file.
    """
    
    with rasterio.open(output_tiff_file, 'w', **tiff_metadata) as dst:
        dst.write(index_array, 1)

In [6]:
def update_index_metadata(band_file, name_index, index_array):
    """
    Update metada from the computed index array
    """
    
   # Reading metadata from 'tiff' file
    with rasterio.open(band_file) as dataset:
        tiff_metadata = dataset.profile

    # Updating metadata from computed index array
    tiff_metadata.update(count=1, dtype=str(index_array.dtype))

    # Saving computed index array to a 'tiff' file
    output_index_tiff = f'outputs/output_{name_index}.tif'
    write_index_to_tiff(index_array, output_index_tiff, tiff_metadata)

In [122]:
# Reading green band (band 3), red band (band 4) and near infrared band (band 8) from 'tiff' files
band3_file = 'sentinel_2/banda_3/T17PNK20230427T155529B03_10m.tif'
band4_file = 'sentinel_2/banda_4/T17PNK20230427T155529B04_10m.tif'
band8_file = 'sentinel_2/banda_8/T17PNK20230427T155529B08_10m.tif'

band_green = readTiffBand(band3_file, 1).astype('float64')
band_red = readTiffBand(band4_file, 1).astype('float64')
band_nir = readTiffBand(band8_file, 1).astype('float64')


# Calculating NDVI
ndvi = calculateNDVI(band_red, band_nir)
update_index_metadata(band8_file, "ndvi", ndvi)


# Calculating NDWI
ndwi = calculateNDWI(band_green, band_nir)
update_index_metadata(band8_file, "ndwi", ndwi)

In [123]:
#Dataset for the calculated ndvi index
df_ndvi = pd.DataFrame(ndvi)
df_ndvi.to_csv("ndvi_index.csv", index=False)
#df_ndvi

In [124]:
#Dataset for the calculated ndwi index
df_ndwi = pd.DataFrame(ndwi)
df_ndwi.to_csv("ndwi_index.csv", index=False)
#df_ndwi